In [1]:
import pickle
from IPython.display import clear_output
from ipywidgets import interact, widgets
from IPython.display import display
from fastai.vision.all import *
from fastai.vision.widgets import *
import re
import time
#SELF SIGNED SSL CERTIFICATE
from OpenSSL import SSL
from cryptography import x509
from cryptography.x509.oid import NameOID
import idna

from socket import socket
from collections import namedtuple
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'OpenSSL'

In [2]:
regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

In [3]:
def check_at_symbol(s):
  if "@" in s:
    return True
  else:
    return False

In [4]:

def check_if_ip(s):
  try:
    s = s.replace("http://","")
    s = s.replace("https://","")
    if s.find('/')!=-1 :
      index = s.find('/')
      s = s[:index]
    x = IP(s)
    return True
  except:
    return False

In [5]:
def check_hexadecimal_code(s):
  if s.find("%")!=-1:
    return True
  else:
    return False

In [6]:
def check_allowed_url_length(s):
  if len(s)>35:
    return True
  else:
    return False

In [7]:
def check_double_slash_symbol(s):
  if s.count("//")>=2:
    return True
  else:
    return False

In [8]:
h_text = {}
h_cookie = {}

In [9]:
def parse_page_from_url(url):
  # page = urlopen(url)
  # html_bytes = page.read()
  # html = html_bytes.decode("utf-8")
  try:
    response = requests.get(url, verify=False)
    # print(response.text)
    html=response.text
    h_text[url] = html
    h_cookie[url] = response.cookie
  except:
    h_text[url] = None
    h_cookie[url] = None

In [10]:
def check_map(url):
  if url in h_text:
    return True
  else:
    parse_page_from_url(url)
    return True

In [11]:
def return_removed_url(s,j):
  return s.replace("."+j,"")

In [12]:
def remove_domain_extensions_from_url(s):
  url_components = s.split(".")
  for i in range(0,len(url_components)):
    for j in range(0,len(domain_extension_list)):
      # print(i,j)
      if url_components[i]==domain_extension_list[j]:
        return return_removed_url(s,domain_extension_list[j])

In [13]:
def check_domain_name(s):
  if s.count("https://")==0:
    s = "https://"+s
  if check_map(s):
    html = h_text[s]
    if html ==None:
      return False
    s_after = remove_domain_extensions_from_url(s)
    # print("s",s)
    if s_after == None:
      # print(s)
      return False
    sa = s_after.split(".")

    for i in range(0,len(sa)):
      if html.find(sa[i])!=-1:
        return True
    return False
  else:
    raise ValueError('Parse Page First')
    

In [14]:
def check_prefix(s):
  return check_domain_name(s)

In [15]:
def check_form(url):
  try:
    if url.count('http://')<=0:
      url = "http://"+url
    check_map(url)
    html = h_text[url]
    soup = BeautifulSoup(html, 'html.parser')
    urls = []
    for link in soup.find_all('form'):
        # link.get('href')
        return True
    return False
  except:
    return False

In [16]:
#READ FROM CSV TO PANDAS DF
domain_extension_list_df = pd.read_csv("domain_extension_list.csv")

In [17]:
domain_extension_list = []
for i in domain_extension_list_df.itertuples():
  # print(i[1])
  domain_extension_list.append(i[1])

In [18]:
famous_url_shorteners = ["goo.gl","bit.ly","owl.ly","deck.ly","bl.ink"]
#abnormal url shortening
def abnormal_url_shortening(url):
  for i in range(0,len(domain_extension_list)):
    # print(domain_extension_list[i])
    dom = str(domain_extension_list[i])
    
    if url.find(dom)!=-1:
      return False
  # print("completed level 1")
  for i in range(0,len(famous_url_shorteners)):
    if url.find(famous_url_shorteners[i])!=-1:
      return False
  return True

In [19]:
def get_all_links_from_url(url):
  reqs = requests.get(url)
  soup = BeautifulSoup(reqs.text, 'html.parser')
  
  urls = []
  for link in soup.find_all('a'):
      print(link.get('href'))
      urls.append(link.get('href'))
      

In [20]:
known_extensions = ["mp4", "m4a", "m4v", "f4v", "f4a", "m4b", "m4r", "f4b", "mov" , "3gp", "3gp", "3g2", "3gpp","mp3","png","jpg","bmp"]
def check_malicious_software_download_extension(url):
  try:
    if url.count("http://")==0:
      url = "http://"+url
    check_map(url)
    html = h_text[url]
    soup = BeautifulSoup(html, 'html.parser')
      
    urls = []
    for link in soup.find_all('a'):
      # print(link.get('href'))
      urls.append(link.get('href'))
      v = link.get('href')
      extension = v.split('.')[-1]
      if len(extension)==3:
        pass
        if extension in known_extensions:
          return False
        else:
          return True
      else:
        return False
  except:
    return False

In [21]:
%%capture
!apt-get install whois

In [22]:
%%capture
!apt -y install netbase

In [23]:
%%capture
!pip install whois

In [24]:
import whois

In [25]:
import datetime
import math

In [26]:
ext=['com', 'uk', 'ac_uk', 'ar', 'at', 'pl', 'be', 'biz', 'br', 'ca', 'cc', 'cl', 'club', 'cn', 'co', 'jp', 'co_jp', 'cz', 'de', 'store', 'download', 'edu', 'education', 'eu', 'fi', 'fr', 'id', 'in_', 'info', 'io', 'ir', 'is_is', 'it', 'kr', 'kz', 'lt', 'ru', 'lv', 'me', 'mobi', 'mx', 'name', 'net', 'ninja', 'se', 'nu', 'nyc', 'nz', 'online', 'org', 'pharmacy', 'press', 'pw', 'rest', 'ru_rf', 'security', 'sh', 'site', 'space', 'tech', 'tel', 'theatre', 'tickets', 'tv', 'us', 'uz', 'video', 'website', 'wiki', 'xyz']

In [27]:
h_whois = {}
def check_whois_map(url):
  
  if url in h_whois:
    return h_whois[url]
  else:
    try:
      # do stuff
      domain=whois.query(url)
      h_whois[url] = domain
    except:
      domain = None
      h_whois[url] = domain
    return domain

In [28]:
def getAge(url):
  url= url.replace("http://","")
  url= url.replace("https://","")
  for i in ext:
    #print(i)
    if url.find(i)!=-1:
      url=url.split('.'+i,1)
      url=url[0]
      url=url+'.'+i
      # print(url)
      break
  domain=check_whois_map(url)
  if domain!=None:
    creationdate=domain.creation_date
    today_Date=datetime.datetime.today()
    day=today_Date.day-creationdate.day
    month=today_Date.month-creationdate.month
    year=today_Date.year-creationdate.year
    age=(year*365)+(month*30)+day
    age=age/365
#     print('age',age)
    return math.floor(age)
  else:
#     print('age',0)
    return 0

In [29]:
def check_age(url):
  try:
    if url.count('http://')==1:
      url = "http://"+url
    age = getAge(url)
    if age < 1:
      # print(url,age)
      return True
    else:
      # print(url,age)
      return False
    
  except:
    return True

In [30]:
def match_port(url):
  for word in url.split(':'):
   if word.isdigit():
      if word!=80:
        return True
  return False

In [31]:
def is_blacklisted(url):
  url= url.replace("http://","")
  url= url.replace("https://","")
  for i in ext:
    #print(i)
    if url.find(i)!=-1:
      url=url.split('.'+i,1)
      url=url[0]
      url=url+'.'+i
      # print(url)
      break
  domain=check_whois_map(url)
  if domain!=None:
    return False
  else:
    return True

In [32]:
%%capture
!pip install pyOpenSSL

In [33]:
HostInfo = namedtuple(field_names='cert hostname peername', typename='HostInfo')

def get_certificate(hostname, port):
    hostname_idna = idna.encode(hostname)
    sock = socket()

    sock.connect((hostname, port))
    peername = sock.getpeername()
    ctx = SSL.Context(SSL.SSLv23_METHOD) # most compatible
    ctx.check_hostname = False
    ctx.verify_mode = SSL.VERIFY_NONE

    sock_ssl = SSL.Connection(ctx, sock)
    sock_ssl.set_connect_state()
    sock_ssl.set_tlsext_host_name(hostname_idna)
    sock_ssl.do_handshake()
    cert = sock_ssl.get_peer_certificate()
    crypto_cert = cert.to_cryptography()
    sock_ssl.close()
    sock.close()

    return HostInfo(cert=crypto_cert, peername=peername, hostname=hostname)

In [34]:
h_ssl_map = {}
def check_if_self_signed_ssl(url):
  if url in h_ssl_map:
    return h_ssl_map[url]
  try:
    # print(url)
    url = url.replace("http://","").replace("https://","").replace("/","")
    cert = get_certificate(url,443)
    if cert.cert.issuer == cert.cert.subject:
      h_ssl_map[url] = True
      return True
    else:
      h_ssl_map[url] = False
      return False
  except:
    h_ssl_map[url] = False
    return False

In [35]:
def check_abnormal_cookie_domain(url):
  try:
    if url.count('http://')<=0:
      url = "http://" + url
    check_map(url)
    p = h_cookie[url]
    # print(p)
    p = str(p)
    # print(p)
    s2 = p.find('for .')
    # print(s2)
    s3 = p.find('/', s2)
    # print(s3)
    p2 = p[s2+5:s3]
    # print(p2)
    if p2 in url:
      return False
    else:
      return True
  except:
    return False

In [36]:
def normalise(value):
  if value == True:
    return 1
  else:
    return 0

In [37]:
path = Path()
filename = 'cybersecuritymodel.sav'
loaded_model = pickle.load(open(path/filename, 'rb'))

In [42]:
url_enter = widgets.Text(
    
    placeholder='Type something',
    description='Enter URL:',
    disabled=False
)
btn_run =widgets.Button(description="Verify")
# lbl_pred = widgets.Label()
out_p1 = widgets.Output()
out_calculating = widgets.Output()
out_calculating_done = widgets.Output()
out_calculating_time = widgets.Output()
out_model_inferencing = widgets.Output()
out_model_inferencing_done = widgets.Output()
out_model_inferencing_time = widgets.Output()
out_model_features = widgets.Output()
out_model_output = widgets.Output()
out_result = widgets.Output()

def on_click_verify(change):
    out_p1.clear_output()
    out_calculating.clear_output()
    out_calculating_done.clear_output()
    out_calculating_time.clear_output()
    out_model_inferencing.clear_output()
    out_model_inferencing_done.clear_output()
    out_model_inferencing_time.clear_output()
    out_model_features.clear_output()
    out_model_output.clear_output()
    out_result.clear_output()
    if re.match(regex, url_enter.value) is None:
        with out_result : display("Enter correct url (add http://) also")
    else:    
        with out_p1 : display(url_enter.value)
        with out_calculating : display("Calculating")
        start_time = time.time()
        lis = get_features(url_enter.value)
        total_time = time.time() - start_time
        with out_calculating : display("Calculating Features Done")
        y1 = "Calculating Features Time : "+str(total_time)
        with out_calculating_time : display(y1)
        with out_calculating : display("Model predict start")
        start_time2 = time.time()
        res = loaded_model.predict(lis)
        with out_calculating : display("Model predict Done")
        total_time2 = time.time() - start_time2
        y2 = "Inference Time : "+str(total_time2)
        with out_calculating_time : display(y2)
        lis_string = str(lis)
        with out_model_features : display(lis)
        res_string = str(res)
        with out_model_output : display(res_string)
#         print(lis)
#         print(res)
        if res[-1] == 0 :
            with out_result : display("Result : Benign Website")
       
        else:
            with out_result : display("Result : Phishing Website")
          
btn_run.on_click(on_click_verify)

In [43]:
def get_features(x):
    f1 = normalise(check_at_symbol(x))
    f2 = normalise(check_if_ip(x))
    f3 = normalise(check_hexadecimal_code(x))
    f4 = normalise(check_allowed_url_length(x))
    f5 = normalise(check_double_slash_symbol(x))
    f6 = normalise(check_domain_name(x))
    f7 = normalise(check_prefix(x))
    f8 = normalise(check_form(x))
    f9 = normalise(abnormal_url_shortening(x))
    f10 = normalise(check_malicious_software_download_extension(x))
    f11 = normalise(check_abnormal_cookie_domain(x))
    f12 = normalise(check_age(x))
    f13 = normalise(match_port(x))
    f14 = normalise(check_if_self_signed_ssl(x))
    f15 = normalise(is_blacklisted(x))
    lis = []
    lis2 = []
    lis2.extend([f1, f2, f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15])
    lis.append(lis2)
    return lis

In [44]:
widgets.VBox([widgets.Label("Enter URL to detect it is phishing URL or not!"),url_enter,btn_run,out_p1,out_calculating,out_calculating_done,out_calculating_time,out_model_inferencing,out_model_inferencing_done,out_model_inferencing_time,out_result])